In [1]:
import pandas as pd

In [2]:
def na_table(x):
    na = pd.DataFrame(x.isna().sum())
    nna = pd.DataFrame(x.notna().sum())
    na.reset_index(inplace=True)
    nna.reset_index(inplace=True)
    na = na[na[0]>0]
    na = na.merge(nna, on='index')
    na.columns = ['col', 'isNA', 'notNA']
    return na

In [3]:
features_dir = '../features/'
# annual_report_info = pd.read_csv(features_dir + 'annual_report_info.csv')
annual_report_info_ = pd.read_csv(features_dir + 'annual_report_info_.csv')
base_info = pd.read_csv(features_dir + 'base_info.csv')
change_info = pd.read_csv(features_dir + 'change_info.csv')
entprise_info = pd.read_csv(features_dir + 'entprise_info.csv')
news_info = pd.read_csv(features_dir + 'news_info.csv')
other_info = pd.read_csv(features_dir + 'other_info.csv')
tax_info = pd.read_csv(features_dir + 'tax_info.csv')
# tax_info_ = pd.read_csv(features_dir + 'tax_info_.csv')

In [4]:
features = base_info.copy()
if 'annual_report_info' in locals().keys():
    features = features.merge(annual_report_info, on='id', how='left')
if 'annual_report_info_' in locals().keys():
    features = features.merge(annual_report_info_, on='id', how='left')
if 'change_info' in locals().keys():
    features = features.merge(change_info, on='id', how='left')
if 'news_info' in locals().keys():
    features = features.merge(news_info, on='id', how='left')
if 'other_info' in locals().keys():
    features = features.merge(other_info, on='id', how='left')
if 'tax_info' in locals().keys():
    features = features.merge(tax_info, on='id', how='left')
    features['tax_cat_'] = features.TAX_AMOUNT.isna()
if 'tax_info_' in locals().keys():
    features = features.merge(tax_info_, on='id', how='left')
    features['tax_cat_'] = features.TAX_AMOUNT.isna()
del features['opscope']

In [5]:
print(features.shape)
na = na_table(features)
na

(24865, 131)


,col,isNA,notNA
0,opto,16040,8825
1,empnum,5250,19615
2,parnum,22526,2339
3,exenum,23487,1378
4,ptbusscope,24865,0
...,...,...,...
106,TAXATION_BASIS,24057,808
107,TAX_RATE,24057,808
108,DEDUCTION,24057,808
109,TAX_AMOUNT,24057,808


In [6]:
threshold = 0.3
col_names = []
for i in (na[na.isNA > threshold * na.shape[0]].col[:-1]):
    col_name = i + '_cat_'
    if col_name not in features.columns:
        col_names.append(col_name)
        features[col_name] = features[i].isna()
features.drop(columns=na[na.isNA==features.shape[0]].col, inplace=True)
open(features_dir + 'cat_cols_expand', 'w').write(str(col_names))

2066

In [7]:
features = features.merge(entprise_info, on='id', how='left')
pd.DataFrame(features[features.label.notna()].label.value_counts())

,label
0.0,13884
1.0,981


In [8]:
features.set_index('id', inplace=True)
features[features.label.notna()].to_csv(features_dir + 'train_test.csv')
features[features.label.isna()].to_csv(features_dir + 'evaluate.csv')